In [1]:
%pip install pymysql

Note: you may need to restart the kernel to use updated packages.


In [2]:
import pymysql

In [3]:
conexao = pymysql.connect(
host="dhauz-instance.cutloqirhpd7.us-east-1.rds.amazonaws.com",
port=3306,
user="candidate_user",
password="D3@bGh664%$1VHv*")

In [4]:
exec = conexao.cursor()
query = """
describe db_hiring_test.raw_transactions_table;
"""
exec.execute(query)
desc = exec.fetchall()
print(desc)


(('IdTransaction', 'varchar(50)', 'YES', '', None, ''), ('AddressOrigin', 'varchar(50)', 'YES', '', None, ''), ('AddressDestination', 'varchar(50)', 'YES', '', None, ''), ('TotalSent', 'varchar(50)', 'YES', '', None, ''), ('Status', 'varchar(50)', 'YES', '', None, ''), ('SentDate', 'varchar(50)', 'YES', '', None, ''), ('ImportDate', 'varchar(50)', 'YES', '', None, ''))


In [5]:
exec = conexao.cursor()
query = """
describe db_hiring_test.raw_transactions_fee;
"""
exec.execute(query)
desc = exec.fetchall()
print(desc)

(('range-start', 'decimal(20,2)', 'YES', '', None, ''), ('range-end', 'decimal(20,2)', 'YES', '', None, ''), ('fee-percentage', 'decimal(20,2)', 'YES', '', None, ''))


In [6]:
# 1. Considerando que a carteira origem é responsável por pagar as taxas de envio, qual carteira
# seria responsável pelo maior pagamento de taxas em janeiro de 2021?
exec = conexao.cursor()
query = """ 
select AddressOrigin, sum(fx.taxa), fx.SentDate from (
    select 
		AddressOrigin,
        (
			select  B.`fee-percentage`
            from  db_hiring_test.raw_transactions_fee as B
            where cast(replace(A.TotalSent, ',', '') as decimal(20,2)) >= B.`range-start` 
            and cast(replace(A.TotalSent, ',', '') as decimal(20,2)) <= B.`range-end` 
		) as taxa,
        cast(replace(A.TotalSent, ',', '') as decimal(20,2)) TotalSent,
        SentDate
    from db_hiring_test.raw_transactions_table as A) fx 
where month(fx.SentDate) = 1 group by fx.AddressOrigin order by sum(fx.taxa) desc limit 1;
"""

exec.execute(query)
desc = exec.fetchall()
print("Carteira com maior pagamento de taxas em janeiro de 2021")
print(desc)

Carteira com maior pagamento de taxas em janeiro de 2021
(('A-62', Decimal('534.00'), '2021-01-01 06:14:33'),)


In [7]:
# 2. E em fevereiro de 2021?
exec = conexao.cursor()
query = """ 
select AddressOrigin, sum(fx.taxa), fx.SentDate from (
    select 
		AddressOrigin,
        (
			select  B.`fee-percentage`
            from  db_hiring_test.raw_transactions_fee as B
            where cast(replace(A.TotalSent, ',', '') as decimal(20,2)) >= B.`range-start` 
            and cast(replace(A.TotalSent, ',', '') as decimal(20,2)) <= B.`range-end` 
		) as taxa,
        cast(replace(A.TotalSent, ',', '') as decimal(20,2)) TotalSent,
        SentDate
    from db_hiring_test.raw_transactions_table as A) fx 
where month(fx.SentDate) = 2 group by fx.AddressOrigin order by sum(fx.taxa) desc limit 1;
"""

exec.execute(query)
desc = exec.fetchall()
print("Carteira com maior pagamento de taxas em fevereiro de 2021")
print(desc)

Carteira com maior pagamento de taxas em fevereiro de 2021
(('A-91', Decimal('34.00'), '2021-02-02 05:41:31'),)


In [8]:
# 3. Qual é o id da transação com a maior taxa paga?
exec = conexao.cursor()
query = """ 
select IdTransaction, sum(fx.taxa) from (
    select 
		IdTransaction,
        (
			select  B.`fee-percentage`
            from  db_hiring_test.raw_transactions_fee as B
            where cast(replace(A.TotalSent, ',', '') as decimal(20,2)) >= B.`range-start` 
            and cast(replace(A.TotalSent, ',', '') as decimal(20,2)) <= B.`range-end` 
		) as taxa,
        cast(replace(A.TotalSent, ',', '') as decimal(20,2)) TotalSent
    from db_hiring_test.raw_transactions_table as A) fx 
group by fx.IdTransaction order by sum(fx.taxa) desc limit 1;
"""

exec.execute(query)
desc = exec.fetchall()
print("id da transação com a maior taxa paga")
print(desc)

id da transação com a maior taxa paga
(('ID1092', Decimal('20.00')),)


In [9]:
# 4. Qual é a média de taxa paga considerando todas as transações realizadas
exec = conexao.cursor()
query = """ 
select avg(fx.taxa) from (
    select 
		IdTransaction,
        (
			select  B.`fee-percentage`
            from  db_hiring_test.raw_transactions_fee as B
            where cast(replace(A.TotalSent, ',', '') as decimal(20,2)) >= B.`range-start` 
            and cast(replace(A.TotalSent, ',', '') as decimal(20,2)) <= B.`range-end` 
		) as taxa,
        cast(replace(A.TotalSent, ',', '') as decimal(20,2)) TotalSent
    from db_hiring_test.raw_transactions_table as A) fx;
"""

exec.execute(query)
desc = exec.fetchall()
print("média de taxa paga")
print(desc)

média de taxa paga
((Decimal('5.878886'),),)
